In [1]:
import csv
import sqlparse
import itertools
from itertools import product, chain

tableSchema= {}

In [2]:
# loading the tabes schema 
def getTableSchema():
    f = open("./files/metadata.txt", "r")
    metaData= f.read()
#     split the metadata based on begin_table
    metaData= metaData.split('<begin_table>\n')
#     split all entries based on end_table to get only the table name and its attributes in each list entry
    metaData= [data.split('<end_table>\n') for data in metaData]
    
#     for each table store its attibute list corresponding to table name after splitting it based on next line char
    for entry in range (1, len(metaData)):
        tableInfo= metaData[entry][0]
        tableInfo= tableInfo.split('\n')[:-1]
        tableSchema[tableInfo[0]]= tableInfo[1:]

In [3]:
def readTable(tableName):
    if(tableName not in tableSchema.keys()):
        print("Invalid Table Name, {}!".format(tableName))
        exit()
    tableFile= open('./files/'+tableName+'.csv', 'r')
    tableReader= csv.reader(tableFile)
    tableData= [[colName for colName in tableSchema[tableName]]]
    for row in tableReader:
        tableData.append(row)    
        
    tableData[1:] = [list(map(int, tableData[tableRow])) for tableRow in range (1, len(tableData))]
#     for row in tableData:
#         print (row)
    return tableData

In [4]:
def prepData(tableList):
#     append the rows of all tables to each other and prepare cross-product dataset
    if(len(tableList)==0):
        print ("Invalid Query..Please provide Table Name(s).")
        exit()
    dataset= readTable(tableList[0])
    for tableNum in range (1, len(tableList)): 
        currentTable= readTable(tableList[tableNum])
        dataset[0]+= currentTable[0]
        finalTable= [dataset[0]]
        for ind1 in range (1, len(dataset)):
            for ind2 in range (1, len(currentTable)):
                finalTable.append(dataset[ind1]+currentTable[ind2])
        dataset= finalTable
    return dataset

In [5]:
def selectAll(tableList):    
#     select all the attirbutes and display the result
    dataset= prepData(tableList)
#     for row in dataset:
#         print (row)
    return dataset

In [6]:
def dropUnwantedAttributes(dataset, attributeList):
    if(len(attributeList)==0):
        print ("Invalid Query..Please provide Attribute Name(s).")
        return 
    
    actualCols= []
    indexNums= []
    for index in range (0, len(dataset[0])):
        attributeName= dataset[0][index]
        actualCols.append(attributeName)
        if(attributeName not in attributeList):
            indexNums.append(index)
#     invalid query if any such attribute is demanded which does not exist
    for attribute in attributeList:
        if attribute not in actualCols:
            print("Invalid Attribute Name, {}!".format(attribute))
            return
#     pop all those attibutes and respective values from all records which are not demanded
    indexNums.sort(reverse= True)    #pop from last index to avoid ambiguities
    for row in dataset:
        for index in indexNums:
            if index < len(row):
                row.pop(index)
#     for row in dataset:
#         print (row)
    return dataset

In [7]:
def dropUnwantedAttributesfromGroups():
    pass

In [8]:
def selectDistinct(dataset):
    tempData= dataset[1:]
    dataset= [dataset[0]]
    for entry in tempData:
        if entry not in dataset:
            dataset.append(entry)
    
    return dataset

In [9]:
# check if the query has aggregate function present
def isAggregate(subQuery):
    if 'max' in subQuery or 'min' in subQuery or 'avg' in subQuery or 'sum' in subQuery or 'count' in subQuery:
        return True
    return False

In [10]:
def solveAggregate(dataset, aggregateAttributes, aggregateFunctions):
#     print (dataset)
    dataset= dropUnwantedAttributes(dataset, aggregateAttributes)#eliminating the name of column
#     if(dataset):
#         dataset = list(chain.from_iterable(dataset)) 
    
    aggregateResults= []
    for function, attribute in zip(aggregateFunctions,aggregateAttributes):
        attributeIndex= dataset[0].index(attribute)
        values= []
        for rowNum in range(1, len(dataset)):
            values.append(dataset[rowNum][attributeIndex])
#         values= list(chain.from_iterable(dataset)) 
        if(function == 'max'):
            aggregateResults.append(max(values))
        elif(function == 'min'):
            aggregateResults.append(min(values))
        elif(function == 'sum'):
            aggregateResults.append(sum(values))
        elif(function == 'avg'):
            aggregateResults.append(sum(values)/len(values))
        else:
            print("Invalid Aggregate function!")
            return
    return aggregateResults

In [11]:
def solveAggregateonGroups():
    pass

In [12]:
# check if the data value in table satisfies the query condition
def isSatisfied(data, operator, value):
    if '<=' == operator:
        return data<=value
    elif '>=' == operator:
        return data>=value
    elif '='== operator:
        return data==value
    elif '<' == operator:
        return data<value
    elif '>' == operator:
        return data>value

In [13]:
# eliminate the records from dataset which do not meet condition specified
def removeRecords(dataset, attribute, operator, value):
    datasetColumns= dataset[0]
    toCheckIndices= []
#     extract the columns with the attribute asked (can be throught multiple columns)
    for colNum in range (0, len(datasetColumns)):
        if datasetColumns[colNum]==attribute:
            print (datasetColumns[colNum])
            toCheckIndices.append(colNum)
    
    if(len(toCheckIndices)==0):
        print("Invalid Attribute Field, please check and try again.")
        exit()
        return
    
    cleanedDataset= [dataset[0]]
    for rowNum in range(1, len(dataset)):
        row= dataset[rowNum]
#         assume that the row satisfies the condition
        toInclude= True
        for col in toCheckIndices:
#             if it does not satisfy make the 'toInclude' flag as False
            if isSatisfied(row[col], operator, value) ==False:
                toInclude= False
                break
#         only if 'toInclude' is still true, add the row to final dataset
        if(toInclude):
            cleanedDataset.append(row)
    
    return cleanedDataset

In [14]:
def union(dataset1, dataset2):
    tuple1=map(tuple,dataset1)
    tuple2=map(tuple,dataset2)
    return list(set(tuple1).union(tuple2))

In [15]:
def intersection(dataset1, dataset2):
    return [value for value in lst1 if value in lst2] 

In [16]:
def meetCondition(dataset, condition):
    if len(condition)==3:        # when there is only 1 condition ['a', '<=', '80']
        dataset= removeRecords(dataset, condition[0], condition[1], int(condition[2]))
        return dataset
    elif len(condition)==7:      # when there are 2 conditions  ['a', '<=', '80', 'AND', 'b', '=','431']
        dataset1= removeRecords(dataset, condition[0], condition[1], int(condition[2]))
        dataset2= removeRecords(dataset, condition[4], condition[5], int(condition[6]))
#         print (dataset1, "\n\n\n\n\n\n\n")
#         print (dataset2, "\n\n\n\n\n\n\n")
        if(condition[3]=='OR'):
            unionDataset= union(dataset1, dataset2)
            return unionDataset
        elif(condition[3]=='AND'):
            intersectionDataset= intersection(dataset1, dataset2)
            return intersectionDataset
        else:
            print("Invalid Condition!")
            exit()
            return
    else:
        print("Invalid Syntax!")
        exit()
        return

In [17]:
def groupBy(dataset, groupbyCol):
    try:
        print ("Group By column= ", groupbyCol)
        groupbyColIndex= dataset[0].index(groupbyCol)
        print("group by index= ", groupbyColIndex)
        # Key function 
        key_func = lambda x: x[groupbyColIndex] 
        groupedDataset= [[dataset[0]]]
        dataset= dataset[1:]

        for key, group in itertools.groupby(dataset, key_func): 
#             print(str(key) + " :", list(group)) 
            groupedDataset.append(list(group)) 
        return groupedDataset
    except:
        print('{} column not in the table.'.format(groupbyCol))
            

In [20]:
def orderBy(dataset, orderbyCol, orderbyStyle):
    if orderbyStyle =='ASC' or orderbyStyle =='asc':
        pass
    elif orderbyStyle =='DESC' or orderbyStyle =='desc':
        pass
    else:
        print("Invalid Ordering Style.")
        return

SyntaxError: invalid syntax (<ipython-input-20-1402bfccc6f3>, line 2)

In [18]:
def processQuery(query):
    if(query[len(query)-1] != ';'): 
        print ("Invalid Syntax..SemiColon Missing!")
        exit()
        return
    
    query= query[:-1]
    parsedQuery = sqlparse.parse(query)[0].tokens
    queryType = sqlparse.sql.Statement(parsedQuery).get_type()
    identifierList = [str(item) for item in sqlparse.sql.IdentifierList(parsedQuery).get_identifiers()]
    print (identifierList)
    
    selectCount= identifierList.count('Select')
    fromCount= identifierList.count('from')
    whereCount, whereClause= 0, ''
    distinctCount= identifierList.count('distinct')
    groupbyCount= identifierList.count('group by')
    orderbyCount= identifierList.count('order by')
    
    if distinctCount==1 and len(identifierList)>5 and 'where' in identifierList[5]:
            whereCount= 1
            whereClause= identifierList[5]
    if distinctCount==0 and len(identifierList)>4 and 'where' in identifierList[4]:
            whereCount= 1
            whereClause= identifierList[4]
                
#     if either select or from clause is absent....and any other has a count greater than 1
    if(selectCount!=1 or fromCount!=1 or whereCount>1 or distinctCount>1):
            print ("Invalid Query!")
        
#    fetch the table names to get the dataset to process queries on
    tableList= identifierList[identifierList.index('from')+1].split(',')
    tableList= [tableName.strip() for tableName in tableList]
    dataset= selectAll(tableList)    #fetch all attributes of table(s)
    
# ------------------------------------------------------------------------------------------------------------ #
    
    if(whereCount ==1):
#         attributeList= identifierList[1].split(',')
#         attributeList= [attributeName.strip() for attributeName in attributeList]
        
#         extract the condition
        condition= whereClause.split(' ')[1:]
        dataset= meetCondition(dataset, condition)   
        
        if groupbyCount==1:
            groupbyCol= identifierList[identifierList.index('group by')+1]
            
            if(isAggregate(identifierList[1])):
#                 perform grouping based on groupbyCol
                dataset= groupBy(dataset, groupbyCol)
        
                attributeList= identifierList[1].split(',')
                attributeList= [attributeName.strip() for attributeName in attributeList]
                
                aggregateCount, aggregateFunction, aggregateAttribute =0, '', ''
                for attributeNum in range (len(attributeList)-1, -1):
                    if(isAggregate(attributeList[attributeNum])):
                        aggregateCount +=1
                        aggregateFunction= attributeList[attributeNum].split('(')[0]
                        aggregateAttribute= attributeList[attributeNum].split('(')[1].split(')')[0]
                        
                if(aggregateCount>1):
                    print ("Invalid Query..Multiple Aggregate Functions are not permissible.")
                else:
                    
                    solveAggregate(tableList, attributeList, aggregateFunction)
                
            else:
                print("Invalid Query..Group By cannot be used without Aggregate Functions on atleast one of the Projected Attribute.")
                
        
    else:
        if groupbyCount==1:
            groupbyCol= identifierList[identifierList.index('group by')+1]
            if(isAggregate(identifierList[1])):
#                 perform grouping based on groupbyCol
                dataset= groupBy(dataset, groupbyCol)
                if dataset:
                    attributeList= identifierList[1].split(',')
                    attributeList= [attributeName.strip() for attributeName in attributeList]
                    
                    aggregateCount, aggregateFunctions, aggregateAttributes =0, [], []
                    for attributeNum in range (len(attributeList)-1, -1):
                        if(isAggregate(attributeList[attributeNum])):
                            aggregateCount +=1
                            aggregateFunctions.append(attributeList[attributeNum].split('(')[0])
                            aggregateAttributes.append(attributeList[attributeNum].split('(')[1].split(')')[0])
                            attributeList[attributeNum]= attributeList[attributeNum].split('(')[1].split(')')[0]
                    
                    dataset= dropUnwantedAttributesfromGroups(dataset, attributeList)                    
                    dataset= solveAggregateonGroups(tableList, attributeList, aggregateFunction)
                    
                if orderbyCount==1:
                    dataset= orderBy(dataset, orderbyCol, orderbyStyle)
                print (dataset)
    
            else:
                print("Invalid Query..Group By cannot be used without Aggregate Functions.")

    
        elif(isAggregate(identifierList[1])):
            attributeList= identifierList[1].split(',')
            attributeList= [attributeName.strip() for attributeName in attributeList]
            aggregateFunctions, aggregateAttributes =[], []
            for attributeNum in range (0, len(attributeList)):
                if(isAggregate(attributeList[attributeNum])):
                    aggregateFunctions.append(attributeList[attributeNum].split('(')[0])
                    aggregateAttributes.append(attributeList[attributeNum].split('(')[1].split(')')[0])
                else:
                    print("Invalid use of Aggregate Functions in Select Clause.")    #queries like select max(A), B are not permissible
                    return
            
            print("aggregate attribute list and functions= ", aggregateAttributes, aggregateFunctions)
            if dataset:
                aggregateResults= solveAggregate(dataset, aggregateAttributes, aggregateFunctions)
                if(aggregateResults==None):
                    return
                elif orderbyCount==1:
                    print("Invalid Query..Order By cannot be used with Aggregate Function.")
                    return 
                else:
                    print(aggregateResults)
        
        elif orderbyCount==1:
            dataset= orderBy(dataset, orderbyCol, orderbyStyle)
            print (dataset)

            
# #        if distinct keyword is present
#         elif(distinctCount==1):
#             print ("Solving the distinct function..")
#             attributeList= identifierList[2].split(',')
#             attributeList= [attributeName.strip() for attributeName in attributeList]
#             dataset= selectAll(tableList)
#             dataset= dropUnwantedAttributes(dataset, attributeList)
#             dataset= selectDistinct(dataset)    
#             print (dataset)
            
#     else:
#         print ("Solving the normal query..")
#         attributeList= identifierList[1].split(',')
#         attributeList= [attributeName.strip() for attributeName in attributeList]
# #             fetch all attributes of table(s)
#         if(len(attributeList)==1 and attributeList[0]=='*'):
#             dataset= selectAll(tableList)
#             print (dataset)
# #             fetch some attribute(s) of table(s)
#         elif(len(attributeList)>=1 and attributeList[0]!='*'):
#             dataset= selectAll(tableList)
#             dataset= dropUnwantedAttributes(dataset, attributeList)
#             print (dataset)
#         else:
#             print("Invalid Query")
#             return

In [19]:
if __name__ == "__main__":
    getTableSchema()
    while(1):
        print("Your Query?")
        query= input()
        processQuery(query)
        print ()

Your Query?
select max(A), sum(B) from table1;
['select', 'max(A), sum(B)', 'from', 'table1']
Invalid Query!
In aggregate functions only
attribute list=  ['max(A)', 'sum(B)']
aggregate attribute list and functions=  ['A', 'B'] ['max', 'sum']
[922, 6187]

Your Query?
select avg(A), min(D) from table1, table2;
['select', 'avg(A), min(D)', 'from', 'table1, table2']
Invalid Query!
In aggregate functions only
attribute list=  ['avg(A)', 'min(D)']
aggregate attribute list and functions=  ['A', 'D'] ['avg', 'min']
[183.0, 85]

Your Query?
select max(A), B from table1;
['select', 'max(A), B', 'from', 'table1']
Invalid Query!
In aggregate functions only
attribute list=  ['max(A)', 'B']
Invalid use of Aggregate Functions in Select Clause.

Your Query?


KeyboardInterrupt: Interrupted by user